In [7]:
############# NOTEBOOK PARAMETERS ##############

parameters = dict()
parameters['file'] = None # set to None for file prompt
parameters['dynamic_range'] = 40
parameters['interpolation'] = 'bilinear'
parameters['cmap'] = 'viridis'

################################################

import os
if not 'module_dir' in globals():
    module_dir = os.path.join(os.getcwd(), '..')
os.chdir(module_dir)

from notebook_functions import *

working_dir: D:\Projects\interaction3\interaction3\analysis\interactive_notebooks\..


In [13]:
## FUNCTIONS ##

def load_data(file):
    
    with closing(sql.connect(file)) as con:
        
        q = 'SELECT frequency, displacement_real, displacement_imag FROM displacements ORDER BY frequency, x, y, z'
        displacements_table = pd.read_sql(q, con)
        
        q = 'SELECT x, y, z, membrane_id, element_id, channel_id FROM nodes ORDER BY x, y, z'
        nodes_table = pd.read_sql(q, con)
    
    freqs = np.unique(np.array(displacements_table['frequency']))
    nfreqs = len(freqs)
    displacements = np.array(displacements_table['displacement_real'] + 1j * displacements_table['displacement_imag'])
    displacements = displacements.reshape((-1, nfreqs), order='F')
    nodes = np.array(nodes_table[['x', 'y', 'z']])
    membrane_ids = np.array(nodes_table['membrane_id'])
    element_ids = np.array(nodes_table['element_id'])
    channel_ids = np.array(nodes_table['channel_id'])
    
    return displacements, freqs, nodes, membrane_ids, element_ids, channel_ids


## LOAD DATA ##

if parameters['file'] is None:
    parameters['file'] = open_file_dialog()

x, freqs, nodes, membrane_ids, element_ids, channel_ids = load_data(parameters['file'])

x_mem = None
nodes_mem = None

# x_mem = data['x_mem']
# x_ch = data['x_ch']
# x_var = np.var(np.abs(x), axis=0)
# x_mem_var = np.var(np.abs(x_mem), axis=0)
# x_ch_var = np.var(np.abs(x_ch), axis=0)

### <center>Notebook parameters</center>

In [ ]:
display(HTML('<center>' + tabulate.tabulate(parameters.items(), tablefmt='html') + '</center>'))

### <center>Plots</center>

In [ ]:
def interpolate_surfaces(meminfo, freqs, f, grid=(20,20)):

    i = np.argmin(np.abs(f - freqs))
    nptx, npty = grid
    surfs = []

    for mem in meminfo:

        x, y, _ = mem['nodes'].T
        x_length = mem['x_length']
        y_length = mem['y_length']

        z = mem['x'][:, i]

        xc = (x.min() + x.max()) / 2
        yc = (y.min() + y.max()) / 2

        xi = np.linspace(xc - x_length / 2, xc + x_length / 2, nptx)
        yi = np.linspace(yc - y_length / 2, yc + y_length / 2, npty)
        zi = griddata((x, y), z, (xi[None, :], yi[:, None]), method='linear', fill_value=0)

        #         zi = np.abs(zi)*np.cos(np.angle(zi))
        zi = np.real(zi)
        xig, yig = np.meshgrid(xi, yi)

        surfs.append((xig, yig, zi))

    return surfs


def surface_plot(x_mem, nodes_mem, freqs, f, cmap='RdBu_r', ax=None, update=False):
    
    i = np.argmin(np.abs(f - freqs))
    nnodes, nmem, nfreq = x_mem.shape
    
    if not update:
        
        for j in range(nmem):
            
            x, y, z = nodes_mem[:, j, i, :].T
            displacement = np.abs(x_mem[:, j, i])
            im = ax.pcolormesh(x, y, displacement, cmap=cmap, shading='gouraud')

        ax.set_axis_off()
        ax.set_aspect('equal')
        
    else:
        
        for im in [a for a in ax.get_children() if isinstance(a, matplotlib.collections.QuadMesh)]:
            im.remove()
            
        for j in range(nmem):
            
            x, y, z = nodes_mem[:, j, i, :].T
            displacement = np.abs(x_mem[:, j, i])
            im = ax.pcolormesh(x, y, displacement, cmap=cmap, shading='gouraud')
            
            
def surface_variance_plot(v, freqs, f, ax=None, lines=None, update=False):

    i = np.argmin(np.abs(f - freqs))
    
    if not update:

        v_line, = ax.plot(freqs/1e6, v, color=blue)
        v_marker, = ax.plot(freqs[i]/1e6, v[i], color=blue, marker='o')

        f_line = ax.axvline(x=freqs[i]/1e6, ls=':')

        lines = [v_line, v_marker, f_line]
        
        ax.set_xlim(0, freqs.max()/1e6)
        ax.set_xlabel('Frequency (MHz)')
        ax.set_ylabel('Displacement variance')
        
        ax.grid('on')
        
        return lines
    
    else:

        v_line, v_marker, f_line = lines
        
        v_marker.set_data(freqs[i]/1e6, v[i])
        f_line.set_xdata(freqs[i]/1e6)


# def onclick(event):
    
#     if event.inaxes in [ax1l, ax1r]:
        
#         f = event.xdata*1e6
        
#         surface_variance_plot(x_var, freqs, ax=ax1, f=f, lines=lines, update=True)
#         surface_plot(header, f=f, ax=ax2, cmap='RdBu_r', update=True)

        
fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(111)

surface_plot(x_mem, nodes_mem, freqs, f, cmap='RdBu_r', ax=ax)

# gs = GridSpec(2,1)
# ax1 = fig.add_subplot(211)
# ax2 = fig.add_subplot(212)
# cid = fig.canvas.mpl_connect('button_press_event', onclick)
# surface_plot(header, f=10e6, ax=ax2, cmap='RdBu_r', update=False)
plt.tight_layout()
